# CS 179 Project

In [ ]:
import pyGM as gm
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
from pyGM.messagepass import LBP, NMF
import pickle

# Load training data and reduce (subsample) if desired

# Read thru file to get numeric ids for each player 
with open('train.csv') as f: lines = f.read().split('\n')
with open('valid.csv') as f: lines2 = f.read().split('\n')
    
p = 0; playerid = {};
for i in range(len(lines)):
  csv = lines[i].split(',');
  if len(csv) != 10: continue;   # parse error or blank line
  player0,player1 = csv[1],csv[4];
  if player0 not in playerid: playerid[player0]=p; p+=1;
  if player1 not in playerid: playerid[player1]=p; p+=1;

nplayers = len(playerid)
playername = ['']*nplayers
for player in playerid: playername[ playerid[player] ]=player;  # id to name lookup

# Making predictions on Validation Data

In [ ]:
nlevels = 15
scale = 0.3

X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)

Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j
        Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))

factors = [ gm.Factor([X[i]], 1.0/nlevels) for i in range(nplayers) ]
pKeep = 0.025  # keep a fraction of the 193k games

for i in range(len(lines)):
    csv = lines[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2 and np.random.rand() < pKeep:
        factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )
        
model = gm.GraphModel(factors)
model.makeMinimal()

lnZ, bel = NMF(model, maxIter=5)

totalGames = 0
correctlyPredictedGames = 0

for i in range(len(lines2)):
    csv = lines2[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        totalGames += 1
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin > 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1

print("Using < or > 0.5: " + str(correctlyPredictedGames / totalGames))

totalGames = 0
correctlyPredictedGames = 0

for i in range(len(lines2)):
    csv = lines2[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        totalGames += 1
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin > np.random.rand() and win == 1) or (predictedWin < np.random.rand() and win == -1)):
            correctlyPredictedGames += 1

print("Using a random value: " + str(correctlyPredictedGames / totalGames))

# Comparing the Number of Games in the Model to Accuracy

In [ ]:
#Testing multiple percentage of games kept
#This will calculate skill level and number of correctly predicted games
emptyList = []
with open('data.pickle', 'wb') as handle:
        pickle.dump(emptyList, handle)

pKeepList = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.20]
for n in pKeepList:
    
    nlevels = 15
    scale = 0.3

    X = [None]*nplayers
    for i in range(nplayers):
        X[i] = gm.Var(i, nlevels)

    Pwin = np.zeros( (nlevels,nlevels) )
    for i in range(nlevels):
        for j in range(nlevels):
            diff = i-j
            Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))
            
    factors = [ gm.Factor([X[i]], 1.0/nlevels) for i in range(nplayers) ]
    
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2 and np.random.rand() < n:
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )
        
    model = gm.GraphModel(factors)
    model.makeMinimal()
    
    lnZ, bel = NMF(model, maxIter=5, verbose=True)
    
    totalGames = 0
    correctlyPredictedGames = 0

    for i in range(len(lines2)):
        csv = lines2[i].split(',')
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            totalGames += 1
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin > 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
                
    pCorrect = correctlyPredictedGames / totalGames
    print(pCorrect)
    
    levels = {}
    for i in range(nplayers):
        levels[playername[i]] = bel[i].table.dot(np.arange(nlevels))
        #print(bel[i].table.dot(np.arange(nlevels)))
        
    data = {'pKeep': n, 'pPredicted': pCorrect, 'SkillLevel': levels}
    with open('data.pickle', 'rb') as handle:
        dataList = pickle.load(handle)
        
    dataList.append(data)
    with open('data.pickle', 'wb') as handle:
        pickle.dump(dataList, handle)

In [ ]:
with open('data.pickle', 'rb') as handle:
    dataList = pickle.load(handle)

pKeep = []
pPredicted = []
for i in range(len(dataList)):
    pKeep.append(dataList[i]['pKeep'])
    pPredicted.append(dataList[i]['pPredicted'])
    
plt.plot(pKeep, pPredicted, 'ro')
plt.show()

# Comparing Number of Skill Levels to Accuracy

In [ ]:
emptyList = []
with open('data2.pickle', 'wb') as handle:
        pickle.dump(emptyList, handle)

nlevels = [2, 5, 10, 15, 20, 25, 30, 40, 50, 60, 75, 100]
pKeep = 0.1
for n in nlevels:
    
    scale = 0.3

    X = [None]*nplayers
    for i in range(nplayers):
        X[i] = gm.Var(i, n)

    Pwin = np.zeros( (n ,n) )
    for i in range(n):
        for j in range(n):
            diff = i-j
            Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))
            
    factors = [ gm.Factor([X[i]], 1.0/n) for i in range(nplayers) ]
    
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2 and np.random.rand() < pKeep:
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )
        
    model = gm.GraphModel(factors)
    model.makeMinimal()
    
    lnZ, bel = NMF(model, maxIter=5, verbose=True)
    
    totalGames = 0
    correctlyPredictedGames = 0

    for i in range(len(lines2)):
        csv = lines2[i].split(',')
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            totalGames += 1
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin > 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
                
    pCorrect = correctlyPredictedGames / totalGames

    levels = {}
    for i in range(nplayers):
        levels[playername[i]] = bel[i].table.dot(np.arange(n))
        #print(bel[i].table.dot(np.arange(nlevels)))
        
    data = {'LevelMax': n, 'pPredicted': pCorrect, 'SkillLevel': levels}
    with open('data2.pickle', 'rb') as handle:
        dataList = pickle.load(handle)
        
    dataList.append(data)
    with open('data2.pickle', 'wb') as handle:
        pickle.dump(dataList, handle)

In [ ]:
with open('data2.pickle', 'rb') as handle:
    dataList = pickle.load(handle)

pKeep = []
pPredicted = []
for i in range(len(dataList)):
    pKeep.append(dataList[i]['LevelMax'])
    pPredicted.append(dataList[i]['pPredicted'])
    
plt.plot(pKeep, pPredicted, 'ro')
plt.xlabel("Max Level")
plt.ylabel("# of Games Correctly Predicted")
plt.title("Effect of Skill Level Scale on Prediction")
plt.show()

# Determining a new player's skill using random games

In [ ]:
# Test how many games are needed for our skill estimation
# to converge to a number
# Use this block to reset the model between tests

# First, create a model and exclude a player (player0) from it
nlevels = 15
scale = 0.3

X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)

Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j
        Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))

factors = [ gm.Factor([X[i]], 1.0/nlevels) for i in range(nplayers) ]
pKeep = 0.1  # keep a fraction of the 193k games
player_left_out = 0  # leave player0 out
player_left_out_games = []  # every game that player0 plays in

for i in range(len(lines)):
    csv = lines[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 == player_left_out or P2 == player_left_out:
        player_left_out_games.append(csv)
        continue
    if P1 != P2 and np.random.rand() < pKeep:
        factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

model = gm.GraphModel(factors)
model.makeMinimal()

lnZ, bel = NMF(model, maxIter=3)

In [ ]:
# This block separates player0's validation games from validation set
players_validation_games = []
players_validation_games_total = 0

for i in range(len(lines2)):
    csv = lines2[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    if P1 == player_left_out or P2 == player_left_out:
        players_validation_games_total += 1
        players_validation_games.append(csv)

In [ ]:
games = [0]
skill_estimation = [bel[player_left_out].table.dot(np.arange(nlevels))]
correctlyPredictedGames = 0
for csv in players_validation_games:
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1
prediction_accuracy = [correctlyPredictedGames / players_validation_games_total]


gap = 100  # add in 100 games at a time

# add in a total of 10 * 100 = 1000 games
# record our skill estimation and validation accuracy for player0 every 100 games
for i in range(10):
    newFactors = []
    for j in range(gap):
        game = player_left_out_games.pop(np.random.randint(len(player_left_out_games)))

        P1,P2 = playerid[game[1]],playerid[game[4]]
        win = -1
        if game[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 == P2:
            j -= 1
            continue
        newFactors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin))
    model.addFactors(newFactors)
    lnZ, bel = NMF(model, maxIter=3)
    
    games.append(games[-1] + gap)
    
    skill_estimation.append(bel[player_left_out].table.dot(np.arange(nlevels)))
    
    correctlyPredictedGames = 0
    for csv in players_validation_games:
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
    prediction_accuracy.append(correctlyPredictedGames / players_validation_games_total)
    
    
data = {"games": games, "skill_estimation": skill_estimation, "prediction_accuracy": prediction_accuracy}
with open('leave_player0_out.pickle', 'wb') as handle:
        pickle.dump(data, handle)

In [ ]:
with open('leave_player0_out.pickle', 'rb') as handle:
    data = pickle.load(handle)

games = data['games']
skill_estimation = data['skill_estimation']
prediction_accuracy = data['prediction_accuracy']
    
plt.plot(games, skill_estimation, 'ro')
plt.xlabel("Games")
plt.ylabel("Skill Estimation")
plt.title("Player0")
plt.show()

In [ ]:
plt.plot(games, prediction_accuracy, 'ro')
plt.xlabel("Games")
plt.ylabel("Prediction Accuracy")
plt.title("Player0")
plt.show()

In [ ]:
games = [0]
skill_estimation = [bel[player_left_out].table.dot(np.arange(nlevels))]
correctlyPredictedGames = 0
for csv in players_validation_games:
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1
prediction_accuracy = [correctlyPredictedGames / players_validation_games_total]


gap = 10  # add in 10 games at a time

# add in a total of 10 * 10 = 100 games
# record our skill estimation and validation accuracyfor player0 every 10 games
for i in range(10):
    newFactors = []
    for j in range(gap):
        game = player_left_out_games.pop(np.random.randint(len(player_left_out_games)))

        P1,P2 = playerid[game[1]],playerid[game[4]]
        win = -1
        if game[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 == P2:
            j -= 1
            continue
        newFactors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin))
    model.addFactors(newFactors)
    lnZ, bel = NMF(model, maxIter=3)
    
    games.append(games[-1] + gap)
    
    skill_estimation.append(bel[player_left_out].table.dot(np.arange(nlevels)))
    
    correctlyPredictedGames = 0
    for csv in players_validation_games:
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
    prediction_accuracy.append(correctlyPredictedGames / players_validation_games_total)
    
data = {"games": games, "skill_estimation": skill_estimation, "prediction_accuracy": prediction_accuracy}
with open('leave_player0_out_2.pickle', 'wb') as handle:
        pickle.dump(data, handle)

In [ ]:
with open('leave_player0_out_2.pickle', 'rb') as handle:
    data = pickle.load(handle)

games = data['games']
skill_estimation = data['skill_estimation']
prediction_accuracy = data['prediction_accuracy']
    
plt.plot(games, skill_estimation, 'ro')
plt.xlabel("Games")
plt.ylabel("Skill Estimation")
plt.title("Player0")
plt.show()

In [ ]:
plt.plot(games, prediction_accuracy, 'ro')
plt.xlabel("Games")
plt.ylabel("Prediction Accuracy")
plt.title("Player0")
plt.show()

In [ ]:
games = [0]
skill_estimation = [bel[player_left_out].table.dot(np.arange(nlevels))]
correctlyPredictedGames = 0
for csv in players_validation_games:
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1
prediction_accuracy = [correctlyPredictedGames / players_validation_games_total]


gap = 2  # add in 2 games at a time

# add in a total of 10 * 2 = 20 games
# record our skill estimation and validation accuracy for player0 every 2 games
for i in range(10):
    newFactors = []
    for j in range(gap):
        game = player_left_out_games.pop(np.random.randint(len(player_left_out_games)))

        P1,P2 = playerid[game[1]],playerid[game[4]]
        win = -1
        if game[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 == P2:
            j -= 1
            continue
        newFactors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin))
    model.addFactors(newFactors)
    lnZ, bel = NMF(model, maxIter=3)
    
    games.append(games[-1] + gap)
    
    skill_estimation.append(bel[player_left_out].table.dot(np.arange(nlevels)))
    
    correctlyPredictedGames = 0
    for csv in players_validation_games:
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
    prediction_accuracy.append(correctlyPredictedGames / players_validation_games_total)
    
data = {"games": games, "skill_estimation": skill_estimation, "prediction_accuracy": prediction_accuracy}
with open('leave_player0_out_3.pickle', 'wb') as handle:
        pickle.dump(data, handle)

In [ ]:
with open('leave_player0_out_3.pickle', 'rb') as handle:
    data = pickle.load(handle)

games = data['games']
skill_estimation = data['skill_estimation']
prediction_accuracy = data['prediction_accuracy']
    
plt.plot(games, skill_estimation, 'ro')
plt.xlabel("Games")
plt.ylabel("Skill Estimation")
plt.title("Player0")
plt.show()

In [ ]:
plt.plot(games, prediction_accuracy, 'ro')
plt.xlabel("Games")
plt.ylabel("Prediction Accuracy")
plt.title("Player0")
plt.show()

# Determining a new player's skill using median player level games

In [ ]:
# Test how many games are needed for our skill estimation
# to converge to a number using median player level games
# Use this block to reset the model between tests

# First, create a model and exclude a player (player0) from it
nlevels = 15
scale = 0.3

X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)

Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j
        Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))

factors = [ gm.Factor([X[i]], 1.0/nlevels) for i in range(nplayers) ]
pKeep = 0.1  # keep a fraction of the 193k games
player_left_out = 0  # leave player0 out
player_left_out_games = []  # every game that player0 plays in

for i in range(len(lines)):
    csv = lines[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 == player_left_out or P2 == player_left_out:
        player_left_out_games.append([P1, P2, win])
        continue
    if P1 != P2 and np.random.rand() < pKeep:
        factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

model = gm.GraphModel(factors)
model.makeMinimal()

lnZ, bel = NMF(model, maxIter=3)

In [ ]:
# sort the games player0 player based on estimated opponent skill level
player_left_out_games.sort(key= lambda x : bel[x[1]].table.dot(np.arange(nlevels)))

In [ ]:
games = [0]
skill_estimation = [bel[player_left_out].table.dot(np.arange(nlevels))]
correctlyPredictedGames = 0
for csv in players_validation_games:
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1
prediction_accuracy = [correctlyPredictedGames / players_validation_games_total]


gap = 100  # add in 100 games at a time

# add in a total of 10 * 10 = 100 games
# record our skill estimation and validation accuracy for player0 every 100 games
for i in range(10):
    newFactors = []
    for j in range(gap):
        #game = player_left_out_games.pop(len(player_left_out_games) // 2)
        game = player_left_out_games.pop()
        P1,P2 = game[0],game[1]
        win = game[2]
        if P1 == P2:
            j -= 1
            continue
        newFactors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin))
    model.addFactors(newFactors)
    lnZ, bel = NMF(model, maxIter=3)
    
    games.append(games[-1] + gap)
    
    skill_estimation.append(bel[player_left_out].table.dot(np.arange(nlevels)))
    
    correctlyPredictedGames = 0
    for csv in players_validation_games:
        if len(csv) != 10: continue
        P1,P2 = playerid[csv[1]],playerid[csv[4]]
        win = -1
        if csv[2]=='[winner]':
            win = 1
        if P1>P2: P1,P2,win=P2,P1,-win
        if P1 != P2:
            predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
            if((predictedWin >= 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
                correctlyPredictedGames += 1
    prediction_accuracy.append(correctlyPredictedGames / players_validation_games_total)
    
data = {"games": games, "skill_estimation": skill_estimation, "prediction_accuracy": prediction_accuracy}
with open('leave_player0_out_sorted.pickle', 'wb') as handle:
        pickle.dump(data, handle)

In [ ]:
with open('leave_player0_out_sorted.pickle', 'rb') as handle:
    data = pickle.load(handle)

games = data['games']
skill_estimation = data['skill_estimation']
prediction_accuracy = data['prediction_accuracy']
    
plt.plot(games, skill_estimation, 'ro')
plt.xlabel("Games")
plt.ylabel("Skill Estimation")
plt.title("Player0 (using best opponent)")
plt.show()

In [ ]:
plt.plot(games, prediction_accuracy, 'ro')
plt.xlabel("Games")
plt.ylabel("Prediction Accuracy")
plt.title("Player0 (using best opponent)")
plt.show()